In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
non_scored_train = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')

## Explore Data

In [ ]:
train_drug

In [ ]:
train_features.shape , train_targets_scored.shape

In [ ]:
test_features.shape

In [ ]:
train_features

In [ ]:
train_targets_scored

In [ ]:
train_targets_scored['5-alpha_reductase_inhibitor'].value_counts()

In [ ]:
sample_submission.head()

In [ ]:
columns = list(train_features.columns)

In [ ]:
# non_scored_train

In [ ]:
test_features

## Data Preprocessing

In [ ]:
train_features.isnull().sum()

In [ ]:
train_targets_scored.isnull().sum()

No NULL values in training Dataset

## Data Split

In [ ]:
train_features.head()

In [ ]:
train_data = train_features.iloc[:,1:]
train_ids = train_features['sig_id']

In [ ]:
train_data['cp_type'].value_counts()

In [ ]:
train_data['cp_dose'].value_counts()

In [ ]:
cat_cp_types = list(train_data['cp_type'].unique())
cat_cp_doses = list(train_data['cp_dose'].unique())

In [ ]:
cat_cp_doses

In [ ]:
cat_cp_types

In [ ]:
map_type = {'trt_cp':0,'ctl_vehicle':1}
map_dose = {'D1':0,'D2':1}

train_data['cp_type'] = train_data['cp_type'].map(map_type)
train_data['cp_dose'] = train_data['cp_dose'].map(map_dose)

In [ ]:
train_data

In [ ]:
test_ids = test_features['sig_id']
test_data = test_features.iloc[:,1:]

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data

In [ ]:
test_data['cp_type'].value_counts()

In [ ]:
test_data['cp_type'] = test_data['cp_type'].map(map_type)
test_data['cp_dose'] = test_data['cp_dose'].map(map_dose)

In [ ]:
test_data

In [ ]:
train_targets_scored

In [ ]:
train_targets = train_targets_scored.iloc[:,1:]
target_cols = list(train_targets.columns)

In [ ]:
train_targets.isnull().sum()

In [ ]:
train_targets.shape

In [ ]:
train_targets.head()

## Create Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_data.shape , train_targets.shape

In [ ]:
model = Sequential()

model.add(Dense(875,activation='relu'))
model.add(Dense(790,activation='relu'))
model.add(Dense(790,activation='relu'))


model.add(Dense(206,activation='softmax'))

model.compile(optimizer='adam',loss='kullback_leibler_divergence',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
model.fit(x=train_data.values,y=train_targets.values,
          epochs=300,
          validation_split=0.33,
         callbacks=[early_stop])

In [ ]:
model.summary()

In [ ]:
predictions = model.predict_proba(test_data)

In [ ]:
predictions[0]

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics

In [ ]:
predicted_result = pd.DataFrame(predictions,columns=target_cols)

In [ ]:
predicted_result

In [ ]:
predicted_result.insert(0,'sig_id',test_ids)

In [ ]:
predicted_result

In [ ]:
predicted_result.to_csv('submission.csv',index=False)